# 🎵 LoFi Music Generator - GPU Training on Colab

**Train your LoFi music AI model with FREE GPU!**

This notebook will:
- ✅ Use Google's FREE GPU (100x faster than CPU)
- ✅ Train on 178k MIDI files from Lakh dataset
- ✅ Save trained model for download
- ✅ Complete in 8-12 hours (not 43 days!)

---

## ⚡ IMPORTANT: Enable GPU First!

1. Click **Runtime** → **Change runtime type**
2. Select **T4 GPU** or **GPU** from Hardware accelerator
3. Click **Save**

**Then run the cells below in order!**

## 📦 Step 1: Setup Environment

In [ ]:
# Check GPU is available
import torch
print(f"🔍 GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("❌ NO GPU! Go to Runtime → Change runtime type → Select GPU")

In [ ]:
# Clone your repository
!git clone https://github.com/andy-regulore/lofi.git
%cd lofi

# Checkout the correct branch
!git checkout claude/add-wav-upload-support-018ZUMDKbSCBaAvMDiN7XXU9

In [ ]:
# Install dependencies
print("📦 Installing dependencies (this takes 3-5 minutes)...")
!pip install -q torch transformers datasets accelerate
!pip install -q miditok miditoolkit pretty_midi
!pip install -q librosa soundfile scipy numpy pandas
!pip install -q pyyaml scikit-learn tqdm tensorboard
print("✅ Dependencies installed!")

## 📂 Step 2: Get Training Data

**Choose ONE option:**
- **Option A:** Download Lakh MIDI Dataset (176k files, ~20GB)
- **Option B:** Upload your own MIDI files from Google Drive

In [ ]:
# OPTION A: Download Lakh MIDI Dataset (recommended)
print("📥 Downloading Lakh MIDI Dataset (~20GB, takes 10-20 minutes)...")
!mkdir -p data/training
!wget -q --show-progress http://hog.ee.columbia.edu/craffel/lmd/lmd_full.tar.gz
print("\n📦 Extracting dataset...")
!tar -xzf lmd_full.tar.gz -C data/training/
!rm lmd_full.tar.gz
print("✅ Dataset ready!")

# Count files
import os
midi_count = sum(1 for root, dirs, files in os.walk('data/training') 
                 for f in files if f.endswith(('.mid', '.midi')))
print(f"\n🎵 Found {midi_count:,} MIDI files")

In [ ]:
# OPTION B: Use Google Drive (skip if you used Option A)
# Uncomment if you have MIDI files in Google Drive

# from google.colab import drive
# drive.mount('/content/drive')

# # Copy from your Google Drive to Colab
# !mkdir -p data/training
# !cp -r /content/drive/MyDrive/your-midi-folder/* data/training/

# # Count files
# import os
# midi_count = sum(1 for root, dirs, files in os.walk('data/training') 
#                  for f in files if f.endswith(('.mid', '.midi')))
# print(f"🎵 Found {midi_count:,} MIDI files")

## 🚀 Step 3: Train the Model!

This will:
1. Tokenize all MIDI files (1-2 hours)
2. Train GPT-2 model (6-10 hours)
3. Save trained model

**Total time: 8-12 hours with GPU**

In [ ]:
# Run training directly in Python (more reliable than scripts)
print("🚀 Starting training...\n")
print("This will take 8-12 hours. You can close this tab and come back later.")
print("Colab will email you when the runtime disconnects (after 12 hours max).\n")

# Make sure we're in the right directory
import os
from pathlib import Path

if not Path('config.yaml').exists():
    print("❌ Error: Not in lofi directory!")
    print("Please run Cell 3 first to clone the repository.")
    print("Then make sure Cell 3 ran: %cd lofi")
    raise FileNotFoundError("config.yaml not found - wrong directory")

print(f"✅ Working directory: {os.getcwd()}\n")

import yaml
import torch
from src.tokenizer import LoFiTokenizer
from src.model import ConditionedLoFiModel
from src.trainer import LoFiTrainer
from sklearn.model_selection import train_test_split

# Load config
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Override settings for Colab
config['training']['output_dir'] = 'models/colab-trained'
config['training']['device'] = 'cuda'
config['training']['fp16'] = True
config['training']['num_epochs'] = 10  # Reduced from 50 to fit in 12 hours
config['training']['batch_size'] = 8  # Larger batch for GPU
config['data']['quality_filters']['require_drums'] = False
config['data']['quality_filters']['min_tempo'] = 1
config['data']['quality_filters']['max_tempo'] = 999

print("="*60)
print("PHASE 1: TOKENIZING MIDI FILES")
print("="*60)

# Initialize tokenizer
tokenizer = LoFiTokenizer(config)
vocab_size = tokenizer.tokenizer.vocab_size
print(f"Vocabulary size: {vocab_size}")

# Find all MIDI files
training_dir = Path('data/training')
midi_files = list(training_dir.glob('**/*.mid')) + list(training_dir.glob('**/*.midi'))
print(f"\nFound {len(midi_files):,} MIDI files")

if len(midi_files) == 0:
    raise ValueError("No MIDI files found! Check data/training directory.")

# Tokenize files
token_sequences = []
success_count = 0
fail_count = 0

print(f"\nTokenizing {len(midi_files):,} files...")
print("This will take 1-2 hours. Progress updates every 1000 files:\n")

for i, midi_file in enumerate(midi_files):
    try:
        # Progress updates
        if (i + 1) % 1000 == 0:
            print(f"  Processed {i+1:,}/{len(midi_files):,} files ({(i+1)/len(midi_files)*100:.1f}%) - "
                  f"Success: {success_count:,}, Failed: {fail_count:,}")
        
        result = tokenizer.tokenize_midi(str(midi_file), check_quality=False)
        if result and 'tokens' in result and len(result['tokens']) > 0:
            chunks = tokenizer.chunk_sequence(result['tokens'])
            if chunks:
                token_sequences.extend(chunks)
                success_count += 1
            else:
                fail_count += 1
        else:
            fail_count += 1
    except Exception as e:
        fail_count += 1
        if fail_count <= 10:
            print(f"  Error with {midi_file.name}: {str(e)[:100]}")
        continue

print(f"\n✅ Tokenization complete!")
print(f"  Success: {success_count:,} files")
print(f"  Failed: {fail_count:,} files")
print(f"  Generated: {len(token_sequences):,} token sequences")

if len(token_sequences) == 0:
    raise ValueError("No valid sequences generated. Check MIDI files.")

print("\n" + "="*60)
print("PHASE 2: SPLITTING DATASET")
print("="*60)

# Split into train/eval
train_sequences, eval_sequences = train_test_split(
    token_sequences, test_size=0.1, random_state=42
)

print(f"Training sequences: {len(train_sequences):,}")
print(f"Evaluation sequences: {len(eval_sequences):,}")

print("\n" + "="*60)
print("PHASE 3: INITIALIZING MODEL")
print("="*60)

# Initialize model
model = ConditionedLoFiModel(config, vocab_size)
model_info = model.get_model_info()

print(f"Model: GPT-2")
print(f"Parameters: {model_info['total_parameters']:,} ({model_info['total_parameters']/1e6:.1f}M)")
print(f"Layers: {model_info['num_layers']}")
print(f"Context length: {model_info['context_length']}")

print("\n" + "="*60)
print("PHASE 4: TRAINING MODEL")
print("="*60)
print("\nThis will take 6-10 hours on GPU.")
print("You can monitor progress with TensorBoard (see next cell).\n")

# Initialize trainer
trainer = LoFiTrainer(model, config, vocab_size)

# Train!
results = trainer.train(train_sequences, eval_sequences)

print("\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print("="*60)

print(f"\nFinal metrics:")
print(f"  Train loss: {results['train_metrics'].get('train_loss', 'N/A')}")
print(f"  Eval loss: {results['eval_metrics'].get('eval_loss', 'N/A')}")
print(f"\nModel saved to: {config['training']['output_dir']}")
print("\n👉 Run the next cell to download your trained model!")

## 📊 Step 4: Monitor Training (Optional)

Run this in a separate cell to check progress

In [ ]:
# Load TensorBoard to monitor training
%load_ext tensorboard
%tensorboard --logdir models/colab-trained/logs

## 💾 Step 5: Download Trained Model

In [ ]:
# Zip the trained model
!zip -r trained_lofi_model.zip models/colab-trained/

# Download to your computer
from google.colab import files
files.download('trained_lofi_model.zip')

print("✅ Model downloaded!")
print("\nUnzip this file and place in your local lofi/models/ directory")

## 🎵 Step 6: Test Generation (Optional)

In [ ]:
# Generate a test track (if scripts exist)
# Note: You can also generate using the web UI after downloading the model

try:
    !python scripts/04_generate.py \
        --config config.yaml \
        --model-path models/colab-trained \
        --output-dir output/test \
        --num-tracks 1 \
        --mood chill \
        --tempo 75

    print("\n✅ Generated test track in output/test/")

    # Download the generated MIDI file
    from google.colab import files
    import os
    if os.path.exists('output/test/midi'):
        midi_files = [f for f in os.listdir('output/test/midi') if f.endswith('.mid')]
        if midi_files:
            files.download(f'output/test/midi/{midi_files[0]}')
    else:
        print("MIDI output directory not found")
except Exception as e:
    print(f"Generation failed: {e}")
    print("You can generate music using the web UI after downloading the model")

## ✅ Next Steps

After training completes:

1. **Download the model** (Step 5 above)
2. **Unzip** on your local machine
3. **Place in** `lofi/models/lofi-gpt2/`
4. **Generate music** using your local web UI!

---

### 🎯 Tips:

- **Colab disconnects after 12 hours** - training will pause. Just re-run the training cell with `--resume` flag
- **Want to continue later?** Mount Google Drive and save checkpoints there
- **Training too long?** Reduce epochs in config.yaml (line 44)

---

### ⚡ Alternative: Reduce Training Size

If you want faster results for testing, edit `config.yaml`:

```yaml
training:
  num_epochs: 10  # Instead of 50
  batch_size: 8   # Increase if you have GPU RAM
```

Or limit MIDI files by only extracting part of the dataset.